In [1]:
# fetch url + title
# https://linuxize.com/post/how-to-extract-unzip-tar-gz-file/
from bs4 import BeautifulSoup
import codecs

In [7]:
import numpy as np
import pandas as pd
import pathlib


def preprocess_soup(soup: BeautifulSoup):
    for node in soup.find_all(["script", "style", "head", "a", "link", "img"]):
        node.decompose()


def extract_description(soup: BeautifulSoup) -> str:
    desc = soup.find("meta", {"name": "description"})
    return desc.attrs.get("content", "") if desc else np.nan


def extract_title(soup: BeautifulSoup) -> str:
    title = soup.find("title")
    header = soup.find("h1")

    if title and header:
        return f"{title.text} {header.text}"

    if title:
        return title.text

    if header:
        return header.text

    return np.nan


def extract_keywords(soup: BeautifulSoup) -> str:
    title = soup.find("meta", {"name": "keywords"})
    return title.attrs.get("content", "") if title else np.nan


def extract_body_chunk(soup: BeautifulSoup, q: float = 0.999) -> str:
    text_chunks = [t.text.strip() for t in soup.find_all(string=True)]
    if not text_chunks:
        return np.nan

    text_sizes = np.asarray([len(x) for x in text_chunks])
    cutoff = np.quantile(text_sizes, q=q)
    return "".join(filter(lambda x: len(x) > cutoff, text_chunks))


def extract_html_features(p: pathlib.Path):
    with codecs.open(p, "r", "utf-8") as f:
        url = f.readline().strip()
        soup = BeautifulSoup(f, "lxml")

    preprocess_soup(soup)

    return dict(
        doc_id=int(p.stem),
        title=extract_title(soup),
        url=url,
        description=extract_description(soup),
        keywords=extract_keywords(soup),
        body=extract_body_chunk(soup),
    )


def make_soup(p: pathlib.Path):
    with codecs.open(p, "r", "utf-8") as f:
        url = f.readline().strip()
        soup = BeautifulSoup(f, "lxml")
        preprocess_soup(soup)
        return soup, url

In [8]:
CONTENT_PATH = "../tmp/content"
html_documents = [f for f in pathlib.Path(CONTENT_PATH).resolve().iterdir()]
html_documents.sort()

html_document_names = [f.stem for f in html_documents]

In [9]:
sample = html_documents[700]
print(sample)
soup, url = make_soup(sample)

/Users/n.teterin/projects/work/notebooks-oneshot/tmp/content/10628.dat


In [10]:
soup.find_all("meta")

[]

In [11]:
extract_html_features(sample)

{'doc_id': 10628,
 'title': 'Как правильно написать реферат',
 'url': 'vocalmuzshcola.ru/muzykalnaya-shkola/stranica-prepodavatelya/kak-pravilno-napisat-referat',
 'description': nan,
 'keywords': nan,
 'body': 'Существуют определенные требования по содержанию реферата. Содержание должно соответствовать названию работы, иметь ценность для публики – освещать достоверные и новые данные, объяснять важные вопросы, иметь доказательную базу. Изложение должно быть ясным, четким и последовательным. Необходимо знать и помнить о таком понятии, как плагиат. Плагиат – это кража контента, дословное копирование текста чужой работы.'}

In [9]:
# https://stackoverflow.com/questions/61860800/running-a-processpoolexecutor-in-ipython
import multiprocessing as mp

mp.set_start_method("fork")

from concurrent import futures
from rich import progress


with futures.ProcessPoolExecutor(max_workers=10) as executor:
    html_features = [
        f
        for f in progress.track(
            executor.map(extract_html_features, html_documents),
            total=len(html_documents),
        )
    ]

Output()

/Users/n.teterin/projects/work/notebooks-oneshot/.venv/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


/Users/n.teterin/projects/work/notebooks-oneshot/.venv/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


/Users/n.teterin/projects/work/notebooks-oneshot/.venv/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


/Users/n.teterin/projects/work/notebooks-oneshot/.venv/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


...

/Users/n.teterin/projects/work/notebooks-oneshot/.venv/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


/Users/n.teterin/projects/work/notebooks-oneshot/.venv/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


/Users/n.teterin/projects/work/notebooks-oneshot/.venv/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


/Users/n.teterin/projects/work/notebooks-oneshot/.venv/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


/Users/n.teterin/projects/work/notebooks-oneshot/.venv/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [11]:
# at first, I forgor to add doc_id to html features, hence had to join
# it with doc_id using url as a reference (thankfully, all URLs are unique)
def extract_url(p: pathlib.Path):
    with codecs.open(p, "r", "utf-8") as f:
        url = f.readline().strip()

    return dict(doc_id=p.stem, url=url)


html_urls = [extract_url(p) for p in html_documents]

In [13]:
df_html_urls = pd.DataFrame(html_urls)
df_html_urls

,doc_id,url
0,1,zrenielib.ru/docs/index-5141.html
1,10,pomogudengami.ru/board/nuzhna_pomoshh/4-259-1
2,100,myfin.by/bank/currency/bobrujsk
3,1000,fb.ru/article/122410/kak-polzovatsya-kompasom-...
4,10000,youtube.com/watch?v=eu0WTLGoSus
...,...,...
28021,9995,lectmania.ru/1x1088b.html
28022,9996,walkspb.ru/articles/neobspb.html
28023,9997,forum.yurclub.ru/index.php?showtopic=107584
28024,9998,stranamam.ru/community/799251


In [18]:
# read titles to ensure I computed features correctly
df_docs_titles = pd.read_csv("../tmp/docs_titles.tsv", delimiter="\t")

In [78]:
df_docs_titles

,doc_id,title
0,15731,ВАЗ 21213 | Замена подшипников ступицы | Нива
1,14829,"Ваз 2107 оптом в Сочи. Сравнить цены, купить п..."
2,15764,Купить ступица Лада калина2. Трансмиссия - пер...
3,17669,Классика 21010 - 21074
4,14852,Ступица Нива — замена подшипника своими руками
...,...,...
27945,16637,Ответы@Mail.Ru: полезно ли кушать творог по ут...
27946,16759,Творог. Полезные свойства и лечение творогом. ...
27947,15358,Творог - Полезные и опасные свойства творога
27948,17287,Ответы@Mail.Ru: Чем полезен творог?


In [126]:
df_html_features = pd.DataFrame(html_features)
df_merged_html_features = df_html_urls.merge(df_html_features, how="inner", on="url")

# ensure we are OK so far
assert len(df_merged_html_features) == len(df_html_features)
assert not df_merged_html_features["url"].isnull().any()

df_merged_html_features["doc_id"] = df_merged_html_features["doc_id"].map(int)
df_merged_html_features["title"] = df_merged_html_features["title"].map(
    # account for NaN values
    lambda x: x.strip() if isinstance(x, str) else x
)
df_merged_html_features["body"] = df_merged_html_features["body"].map(
    # account for NaN values
    lambda x: x.strip() if isinstance(x, str) else x
)

df_final_html_features = (
    df_merged_html_features
    # description was completely empty by mistake, but one can rerun
    # the processing loop and obtain proper descriptions
    .drop(columns=["description"])
    .merge(df_docs_titles[["doc_id"]], how="inner", on="doc_id")
    .sort_values(by="doc_id")
    .set_index("doc_id")
    .reset_index()
)
df_final_html_features

,doc_id,url,title,keywords,body
0,1,zrenielib.ru/docs/index-5141.html,М. Б. Аншина Центр репродукции и генетики «Фер...,NaN,Симптомы эндокринных заболеваний можно раздели...
1,2,kak-perevesti-online.ru/perevody-qiwi-wallet.html,Переводы Киви кошелька,NaN,Активный выбор людей в пользу безналичного рас...
2,3,timecops.biz/forum/viewtopic.php?f=13&t=319,ПРОЕКТ ПАТРУЛИ ВРЕМЕНИ - РЕАБИЛИТАЦИЯ ДУХОВНЫХ...,NaN,"нологией, которую мы сможем применять. И единс..."
3,4,proffi95.ru/blogs/prepodavanie-v-nachalnyh-kla...,→ Блог,NaN,"Творческая, ищущая мысль направляет свои усили..."
4,5,xn----jtbaaldsgaoflxr4fyc.xn--p1ai/%D0%BD%D0%B...,Как быстро понизить холестерин. Высокий холест...,NaN,"У меня ""плохой"" холестерин 6,72. Я испугалась,..."
...,...,...,...,...,...
27945,28022,jplant.ru/bolezni-i-vrediteli/listya-anturiuma...,,NaN,Однако и листья антуриума не менее притягатель...
27946,28023,catalogcars.net/toyota/toyota-hilux-surf-3-0-t...,,NaN,В сумму которая была на руках что-то как-то ни...
27947,28024,prostoprikol.com/video/cTltRfRTPnk,How to download power director 14 full version...,NaN,"HERE YOU GO, NOW YOU GET A WAY TO DOWNLOAD TH..."
27948,28025,sector-book.ru/items/skachat_proigryvatel_avi_...,Скачать проигрыватель avi iphone,NaN,фантастика\n смотреть фантастику\n фантастика ...


In [141]:
# last sanity check
assert set(df_final_html_features["doc_id"]) == set(df_docs_titles["doc_id"])

# feather is more lightweight than csv and does not have encoding/quotation issues since it is a binary format
df_final_html_features[["doc_id", "url", "title", "keywords", "body"]].to_feather(
    "../tmp/docs_features.feather"
)

In [142]:
!du -h ../tmp/docs_features.feather

146M	../tmp/docs_features.feather


In [143]:
df_reconstructed = pd.read_feather("../tmp/docs_features.feather")

In [144]:
df_reconstructed

,doc_id,url,title,keywords,body
0,1,zrenielib.ru/docs/index-5141.html,М. Б. Аншина Центр репродукции и генетики «Фер...,None,Симптомы эндокринных заболеваний можно раздели...
1,2,kak-perevesti-online.ru/perevody-qiwi-wallet.html,Переводы Киви кошелька,None,Активный выбор людей в пользу безналичного рас...
2,3,timecops.biz/forum/viewtopic.php?f=13&t=319,ПРОЕКТ ПАТРУЛИ ВРЕМЕНИ - РЕАБИЛИТАЦИЯ ДУХОВНЫХ...,None,"нологией, которую мы сможем применять. И единс..."
3,4,proffi95.ru/blogs/prepodavanie-v-nachalnyh-kla...,→ Блог,None,"Творческая, ищущая мысль направляет свои усили..."
4,5,xn----jtbaaldsgaoflxr4fyc.xn--p1ai/%D0%BD%D0%B...,Как быстро понизить холестерин. Высокий холест...,None,"У меня ""плохой"" холестерин 6,72. Я испугалась,..."
...,...,...,...,...,...
27945,28022,jplant.ru/bolezni-i-vrediteli/listya-anturiuma...,,None,Однако и листья антуриума не менее притягатель...
27946,28023,catalogcars.net/toyota/toyota-hilux-surf-3-0-t...,,None,В сумму которая была на руках что-то как-то ни...
27947,28024,prostoprikol.com/video/cTltRfRTPnk,How to download power director 14 full version...,None,"HERE YOU GO, NOW YOU GET A WAY TO DOWNLOAD TH..."
27948,28025,sector-book.ru/items/skachat_proigryvatel_avi_...,Скачать проигрыватель avi iphone,None,фантастика\n смотреть фантастику\n фантастика ...
